In [1]:
import proteinmpnn.run
import proteinmpnn.utils.misc

from proteinmpnn.run import load_protein_mpnn_model, design_sequences, untokenise_sequence
from proteinmpnn.utils.misc import find_files

# Manual mode (more control)

In [2]:
model = load_protein_mpnn_model(model_type="ca")

2023-07-16 21:11:43,601 INFO:
	Number of edges: 48 [in load_protein_mpnn_model at run.py:100]
2023-07-16 21:11:43,601 INFO:
	Training noise level: 0.2A [in load_protein_mpnn_model at run.py:102]
2023-07-16 21:11:43,601 INFO:
	Training noise level: 0.2A [in load_protein_mpnn_model at run.py:102]


In [3]:
backbones = find_files("./data", ".pdb")  # Or numpy or torch objects

2023-07-16 21:11:43,681 INFO:
	Found 2 files with extension .pdb in ./data. [in find_files at misc.py:43]


In [4]:
samples = design_sequences(backbones, model, num_seq_per_target=3, compute_scores=True)

In [5]:
untokenise_sequence(samples[0]["S"])

'MLPPEEQLALDFIKALEKADPELAKKVVGEDTELEVNGKKFKGEEIVEFVKELKEKGVKFKLKSWRWEGDKLVFDMEVEKDGKKEKVKVTITVKDGKIAKIVIEKK'

In [6]:
samples[0]["prob"]

tensor([0.3781])